## Mecab 사용자 사전 만드는 프로그램


In [1]:
import pandas as pd
import numpy as np

#정규식을 위한 라이브러리
import re

#### dataset 설명
1. keyword_dic.csv
  - 수집된 논문 정보 중 주제어를 처리하여 한글 단어를 추출한 목록 파일
2. tta_dic.csv 
  - TTA정보통신용어사전(word.tta.or.kr)을 csv로 만든 파일로, 용어, 동의어, 설명으로 구성됨

In [2]:
words = pd.read_csv("./dataset/keyword_dic.csv")

In [3]:
words.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105684 entries, 0 to 105683
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   keyword  105684 non-null  object
dtypes: object(1)
memory usage: 825.8+ KB


* TTA 용어인 경우에만 수행하는 로직 (시작)
----

In [ ]:
# TTA 용어사전은 용어가 한글, 영어, 한문 등으로 섞여 있고, 숫자, "," 등 의 문자가 혼재되어 있어
# 최소한의 작업을 위해 한글만 추출할 수 있도록 정규식을 사용

# 논문 주제어는 한글 용어를 추출하였으므로 해당 작업은 불필요함

def search_word_for_tta(x) :
    p = re.compile(".*[ㄱ-ㅎㅏ-ㅣ가-힣]+[+|#]{0,2},|.*[ㄱ-ㅎㅏ-ㅣ가-힣]+[\s|-]?\w{0,5}[.|-]?\w{0,5}[+|#]{0,2},|.*[(]?[ㄱ-ㅎㅏ-ㅣ가-힣|\w]+\s?[)]?[\s|-]?\w{0,5}[.|-]?\w{0,5}[+|#]{0,2},")
    match = p.search(x)
    if match != None :
        word = match.group()
        return word[:-1]
    else :
        return None

In [ ]:
words['keyword'] = words['용어'].apply(lambda x : search_word_tta(x))

-----
* TTA 용어인 경우에만 수행하는 로직 (종료)

In [4]:
words.head()

,keyword
0,내연기관
1,에너지
2,환경
3,오염물질
4,수소 연료전지 자동차


#### Mecab 사용자 사전 설명

[Mecab 사전형식 링크 (클릭하세요)](https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=1, "Mecab 사전형식 링크")

In [5]:
# 종성 유무(T, F)를 표현하기 위해 마지막 문자의 받침 유무를 확인
def checkTrait(c):
    return (int((ord(c) - 0xAC00) % 28) != 0)

In [6]:
tags = []

for word in words['keyword'].tolist() :
    line  = [word, '', '', '', 'NNG', '*', checkTrait(word[-1]) and 'T' or 'F', word, '*', '*', '*', '*', '*']
    tags.append(line) 

In [7]:
df = pd.DataFrame(tags)
df.head()
df.to_csv('./result/nnp_user.csv', encoding='utf-8', index=False, header=False)

### Mecab 사전 설치
1. 만들어진 ./result/nnp_user.csv 파일의 내용을 **C:\mecab\user-dic\nnp.csv** 파일에 복사
2. 아래 화면과 같이 PowerShell을 실행 후 mecab 디렉토리로 이동하여 스크립트 실행
  - 실행 전    
  ![PowerShell] (./images/1.powershell.png)
  - 실행 후    
  ![PowerShell] (./images/2.powershell.png)
  - 설치 확인 (아래 그림과 같이 해당 사전디렉토리에 user-nnp.csv 파일이 생겼다면 성공)    
  ![PowerShell] (./images/3.powershell.png)

In [8]:
from eunjeon import Mecab
mecab = Mecab("c://mecab//mecabrc")

#./dataset/keword_dic.csv의 72번째 용어
mecab.pos("연료전지 자동차 가상 샤시 플랫폼")

[('연료전지 자동차 가상 샤시 플랫폼', 'NNG')]